In [6]:
import os
import sqlite3
import time
import requests
import json


In [ ]:
# do this to get the first token 

# goto:
# https://www.strava.com/oauth/authorize?client_id=121298&response_type=code&redirect_uri=http://localhost&approval_prompt=force&scope=activity:read_all



# get auth code from resulting url
# http://localhost/?state=&code=236616d076eecdbecd4147be329c46cefd1b2cd5&scope=read,activity:read_all


client_id = '121298'
client_secret = '003ac5e5560276a0a04412a5940d60aac9e0a6db'
# auth_code = '223dd38817627ac8aa3125a199bf41a6422ab87c'

# response = requests.post(
#     url="https://www.strava.com/oauth/token",
#     data={
#         'client_id': client_id,
#         'client_secret': client_secret,
#         'code': auth_code,
#         'grant_type': 'authorization_code'
#     }
# )

# tokens = response.json()

# access_token = tokens['access_token']
# refresh_token = tokens['refresh_token']



In [72]:
# do this to refresh the token

response = requests.post(
    'https://www.strava.com/api/v3/oauth/token',
    data={
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }
)

tokens = response.json()
access_token = tokens['access_token']
refresh_token = tokens['refresh_token']
headers = {
    'Authorization': f'Bearer {access_token}'
}

In [79]:
# check usage - still do not know when things reset
response = requests.get(
    'https://www.strava.com/api/v3/athlete',
    headers={'Authorization': f'Bearer {access_token}'}
)

print("Status:", response.status_code)
print("Rate Limit:", response.headers.get('X-RateLimit-Limit'))
print("Rate Usage:", response.headers.get('X-RateLimit-Usage'))



Status: 429
Rate Limit: 200,2000
Rate Usage: 7,1702


In [78]:
page = 2
response = requests.get(
        f"https://www.strava.com/api/v3/athlete/activities?page={page}&per_page=1",
        headers=headers
    )
data = response.json()
data

{'message': 'Rate Limit Exceeded',
 'errors': [{'resource': 'Application',
   'field': 'read rate limit',
   'code': 'exceeded'}]}

In [27]:
# make the db and functions
DB_FILE = 'strava_cache.db'
# === SETUP DATABASE ===
conn = sqlite3.connect(DB_FILE)
c = conn.cursor()

# Create activity table
c.execute('''
CREATE TABLE IF NOT EXISTS activities (
    id INTEGER PRIMARY KEY,
    data TEXT
)
''')

# Create stream table
c.execute('''
CREATE TABLE IF NOT EXISTS streams (
    activity_id INTEGER PRIMARY KEY,
    data TEXT
)
''')

conn.commit()

# === FUNCTIONS ===

def activity_cached(activity_id):
    c.execute('SELECT 1 FROM activities WHERE id = ?', (activity_id,))
    return c.fetchone() is not None

def stream_cached(activity_id):
    c.execute('SELECT 1 FROM streams WHERE activity_id = ?', (activity_id,))
    return c.fetchone() is not None

def cache_activity(activity_id, data):
    c.execute('INSERT OR REPLACE INTO activities (id, data) VALUES (?, ?)', (activity_id, json.dumps(data)))
    conn.commit()

def cache_stream(activity_id, data):
    c.execute('INSERT OR REPLACE INTO streams (activity_id, data) VALUES (?, ?)', (activity_id, json.dumps(data)))
    conn.commit()

In [58]:
# fetch functions
def fetch_activities(n):
    activities = []
    page = 1
    per_page = 100

    while len(activities) < n:
        response = requests.get(
            f'https://www.strava.com/api/v3/athlete/activities',
            headers=headers,
            params={'per_page': per_page, 'page': page}
        )
        time.sleep(1)  # respect rate limits
        if response.status_code != 200:
            print("Error fetching activities:", response.text)
            break

        page_data = response.json()
        if not page_data:
            break

        activities.extend(page_data)
        page += 1

    return activities[:n]

def fetch_stream(activity_id):
    response = requests.get(
        f'https://www.strava.com/api/v3/activities/{activity_id}/streams',
        headers=headers,
        params={"keys": "latlng", "key_by_type": "true"}
    )
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to get stream for activity {activity_id}: {response.status_code}")
        return None

In [59]:
# I think I can run this to get and cache all of the activities in a database - still might have some issues with the rate...


NUM_ACTIVITIES_TO_FETCH = 50

print(f"Fetching up to {NUM_ACTIVITIES_TO_FETCH} activities...")
activities = fetch_activities(NUM_ACTIVITIES_TO_FETCH)
print(f"Found {len(activities)} activities")

for act in activities:
    act_id = act['id']
    if not activity_cached(act_id):
        cache_activity(act_id, act)
        time.sleep(1)  # respect rate limits
        print(f"Cached activity {act_id}")
    else:
        print(f"Activity {act_id} already cached")

    if not stream_cached(act_id):
        stream = fetch_stream(act_id)
        if stream and 'latlng' in stream:
            cache_stream(act_id, stream)
            print(f"Cached stream for {act_id}")
            time.sleep(1)  # respect rate limits
    else:
        print(f"Stream for {act_id} already cached")

print("Done.")
conn.close()


Fetching up to 50 activities...
Error fetching activities: {"message":"Rate Limit Exceeded","errors":[{"resource":"Application","field":"read rate limit","code":"exceeded"}]}
Found 0 activities
Done.


{'message': 'Rate Limit Exceeded',
 'errors': [{'resource': 'Application',
   'field': 'read rate limit',
   'code': 'exceeded'}]}

In [14]:
import folium
from folium.plugins import HeatMap

with open('data.json', 'r') as f:
    all_coords = json.load(f)





m = folium.Map(location=all_coords[0], zoom_start=12)
HeatMap(all_coords, 
        radius=4,
        blur=2,
        min_opacity=0.5,
        max_zoom=18,).add_to(m)



m.save('strava_heatmap.html')


In [15]:
with open('data.json', 'r') as f:
    all_coords = json.load(f)



m = folium.Map(location=[51.5, -0.1], zoom_start=12)  # adjust center as needed
    
folium.PolyLine(
    all_coords, 
    color="blue", 
    weight=2,  # thinner lines
    opacity=0.7
).add_to(m)

m.save('sharp_map.html')


True